In [34]:
import numpy as np
from scipy.optimize import minimize


# Define the likelihood function
def likelihood(params, data):
    o1, o2, a1, a2, b1, b2 = params
    X1, Y1, Z1, W1, X2, Y2, Z2, W2 = data
    
    tot1 = X1 + Y1 + Z1 + W1
    tot2 =  X2 + Y2 + Z2 + W2
    tot = tot1 + tot2

    p1 = (o1 * (1-a1) * (1-b1) + (1-o1) * a1 * b1)
    p2 = (o1 * (1-a1) * b1 + (1-o1) * a1 * (1-b1))
    p3 = (o1 * a1 * (1-b1) + (1-o1) * (1-a1) * b1)
    p4 = (o1 * a1 * b1 + (1-o1) * (1-a1) * (1-b1))
    p5 = (o2 * (1-a2) * (1-b2) + (1-o2) * a2 * b2)
    p6 = (o2 * (1-a2) * b2 + (1-o2) * a2 * (1-b2))
    p7 = (o2 * a2 * (1-b2) + (1-o2) * (1-a2) * b2)
    p8 = (o2 * a2 * b2 + (1-o2) * (1-a2) * (1-b2))
    eps = 1e-10
    L = (
        (p1 + eps)**(X1/tot) *
        (p2 + eps)**(Y1/tot) *
        (p3 + eps)**(Z1/tot) *
        (p4 + eps)**(W1/tot) *
        (p5 + eps)**(X2/tot) *
        (p6 + eps)**(Y2/tot) *
        (p7 + eps)**(Z2/tot) *
        (p8 + eps)**(W2/tot)
    )

    return -np.log(L) # Return the negative log-likelihood for minimization


# Observed data: X1, Y1, Z1, W1, X2, Y2, Z2, W2
data = (14, 4, 9, 528, 887, 31, 37, 367)

# Initial parameter guesses: o1, o2, a1, a2, b1, b2
init_params = [0.5, 0.5, 0.2, 0.2, 0.2, 0.2]

# Parameter bounds
bounds = [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]

# Minimize the negative log-likelihood
result = minimize(likelihood, init_params, args=(data,), bounds=bounds)

# Get the maximum likelihood estimates for the parameters
o1_mle, o2_mle, b1_mle, b2_mle, a2_mle, a1_mle = result.x

print("Maximum Likelihood Estimates:")
print(f"a1: {a1_mle:.4f}, b1: {b1_mle:.4f}, o1: {o1_mle:.4f}")
print(f"a2: {a2_mle:.4f}, b2: {b2_mle:.4f}, o2: {o2_mle:.4f}")

Maximum Likelihood Estimates:
a1: 0.0209, b1: 0.0071, o1: 0.0257
a2: 0.0166, b2: 0.0318, o2: 0.7076


![alternative text](figs/orginal_hw_paper.png)

In [4]:
import numpy as np

def likelihood(params, data):
    o1, o2, a1, a2, b1, b2 = params
    X1, Y1, Z1, W1, X2, Y2, Z2, W2 = data
    
    tot1 = X1 + Y1 + Z1 + W1
    tot2 =  X2 + Y2 + Z2 + W2
    tot = tot1 + tot2

    p1 = (o1 * (1-a1) * (1-b1) + (1-o1) * a1 * b1)
    p2 = (o1 * (1-a1) * b1 + (1-o1) * a1 * (1-b1))
    p3 = (o1 * a1 * (1-b1) + (1-o1) * (1-a1) * b1)
    p4 = (o1 * a1 * b1 + (1-o1) * (1-a1) * (1-b1))
    p5 = (o2 * (1-a2) * (1-b2) + (1-o2) * a2 * b2)
    p6 = (o2 * (1-a2) * b2 + (1-o2) * a2 * (1-b2))
    p7 = (o2 * a2 * (1-b2) + (1-o2) * (1-a2) * b2)
    p8 = (o2 * a2 * b2 + (1-o2) * (1-a2) * (1-b2))
    eps = 1e-10
    L = (
        (p1 + eps)**(X1/tot) *
        (p2 + eps)**(Y1/tot) *
        (p3 + eps)**(Z1/tot) *
        (p4 + eps)**(W1/tot) *
        (p5 + eps)**(X2/tot) *
        (p6 + eps)**(Y2/tot) *
        (p7 + eps)**(Z2/tot) *
        (p8 + eps)**(W2/tot)
    )

    return -np.log(L) # Return the negative log-likelihood for minimization

# Metropolis-Hastings update
def mh_update(curr_params, data, proposal_sd):
    # Draw a proposal
    prop_params = np.random.normal(loc=curr_params, scale=proposal_sd)
    prop_params = np.clip(prop_params, 0, 1) # Ensure parameters stay within bounds
    
    # Calculate acceptance probability
    curr_ll = likelihood(curr_params, data)
    prop_ll = likelihood(prop_params, data)
    accept_prob = min(1, np.exp(curr_ll - prop_ll))
    #print(accept_prob)
    
    # Accept or reject proposal
    if np.random.uniform() < accept_prob:
        return prop_params
    else:
        return curr_params

# Gibbs sampler
def gibbs_sampler(data, init_params, num_iters, proposal_sd):
    params = init_params
    
    # Store history of samples
    param_samples = np.zeros((num_iters, len(init_params)))
    
    # Perform Gibbs updates
    for i in range(num_iters):
        params = mh_update(params, data, proposal_sd)
        param_samples[i] = params
    return param_samples

#Observed data: X1, Y1, Z1, W1, X2, Y2, Z2, W2
data = (14, 4, 9, 528, 887, 31, 37, 367)

#Initial parameter guesses: o1, o2, a1, a2, b1, b2
init_params = np.array([0.5, 0.5, 0.2, 0.2, 0.2, 0.2])

#Run Gibbs sampler
param_samples = gibbs_sampler(data, init_params, num_iters=1000, proposal_sd=0.01)

#Calculate mean parameter estimates
mean_params = param_samples.mean(axis=0)

print("Mean Parameter Estimates:")
print(f"o1: {mean_params[0]:.4f}, o2: {mean_params[1]:.4f}, a1: {mean_params[2]:.4f}, a2: {mean_params[3]:.4f}, b1: {mean_params[4]:.4f}, b2: {mean_params[5]:.4f}")

Mean Parameter Estimates:
o1: 0.5877, o2: 0.5870, a1: 0.1792, a2: 0.1206, b1: 0.1573, b2: 0.2077
